In [1]:
from google.transit import gtfs_realtime_pb2
import urllib
import datetime
import time
import pandas as pd
import numpy as np
import csv

In [2]:
def write_header(columns_names, file_name):
    with open(file_name, mode='w') as file:
        wr = csv.writer(file, dialect='excel')
        wr.writerow(columns_names)

In [3]:
header_updates = [
    "feed_time",
    "trip_id",
    "vehicle_id",
    "start_time",
    "start_date",
    "route_id",
    "direction_id",
    "stop_id",
    "stop_sequence",
    "schedule_relationship",
    "arrival_time",
    "arrival_uncertainty",
    "departure_time",
    "depature_uncertainty",
]

write_header(header_updates, "trip_updates.txt")

header_positions = [
    "feed_time",
    "trip_id",
    "vehicle.id",
    "start_time",
    "start_date",
    "route_id",
    "direction_id",
    "schedule_relationship",
    "latitude",
    "longitude",
    "bearing",
    "stop_id",
    "current_stop_sequence",
    "current_status",
    "timestamp",
]

write_header(header_positions, "vehicle_positions.txt")

In [4]:
def get_feed(url):
    feed = gtfs_realtime_pb2.FeedMessage()
    response = urllib.request.urlopen(url)
    feed.ParseFromString(response.read())
    return feed

In [5]:
def write_feed_to_csv(feed, file_name, is_position, ts):
    file = open(file_name, mode="a")
    wr = csv.writer(file, dialect="excel")

    if is_position:
        for item in feed.entity:
            vehicle = item.vehicle
            record = [
                ts,
                vehicle.trip.trip_id,
                vehicle.vehicle.id,
                vehicle.trip.start_time,
                vehicle.trip.start_date,
                vehicle.trip.route_id,
                vehicle.trip.direction_id,
                vehicle.trip.schedule_relationship,
                vehicle.position.latitude,
                vehicle.position.longitude,
                vehicle.position.bearing,
                vehicle.stop_id,
                vehicle.current_stop_sequence,
                vehicle.current_status,
                vehicle.timestamp,
            ]
            wr.writerow(record)
    else:
        for trip in feed.entity:
            for stop in trip.trip_update.stop_time_update:
                record = [
                    ts,
                    trip.id,
                    trip.trip_update.vehicle.id,
                    trip.trip_update.trip.start_time,
                    trip.trip_update.trip.start_date,
                    trip.trip_update.trip.route_id,
                    trip.trip_update.trip.direction_id,
                    stop.stop_id,
                    stop.stop_sequence,
                    stop.schedule_relationship,
                    stop.arrival.time,
                    stop.arrival.uncertainty,
                    stop.departure.time,
                    stop.departure.uncertainty,
                ]
                wr.writerow(record)
    file.close()

In [6]:
url_positions = 'https://cdn.mbta.com/realtime/VehiclePositions.pb'
url_updates = 'https://cdn.mbta.com/realtime/TripUpdates.pb'

In [ ]:
while True:
    feed_positions = get_feed(url_positions)
    feed_updates = get_feed(url_updates)
    ts = datetime.datetime.now().timestamp()
    write_feed_to_csv(
        feed=feed_positions, file_name="vehicle_positions.txt", is_position=True, ts=ts
    )

    write_feed_to_csv(
        feed=feed_updates, file_name="trip_updates.txt", is_position=False, ts=ts
    )
    time.sleep(15)